# Import Libs

In [1]:
import pandas as pd
import numpy as np
from statistics import mean
from tqdm import tqdm
import matplotlib.pyplot as plt

from nltk import word_tokenize
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer

from sklearn.model_selection import StratifiedShuffleSplit, GridSearchCV

import tensorflow as tf
from tensorflow.keras.utils import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Embedding, Dense, Dropout
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import EarlyStopping
from scikeras.wrappers import KerasClassifier

from statistics import mean
from sklearn.metrics import classification_report, f1_score, accuracy_score

# Load dataset

In [2]:
# pre_processed_text = 'gemini_embedding'
# pre_processed_text = 'text_embed'
pre_processed_text = 'pre_processed_text'

In [3]:
df = pd.read_csv('./dataset/hsd_pre_processed.csv')

In [4]:
df

,text,hatespeech_comb,hatespeech_G1,annotator_G1,hatespeech_G2,annotator_G2,hatespeech_G3,annotator_G3,pre_processed_text
0,@__andrea__b \nO cara vive em outro mundo\nNão...,1,1,A,1.0,V,0,E,cara vive outro mundo mundo real refugiados vi...
1,@_carmeloneto Estes incompetentes não cuidam n...,0,1,D,0.0,V,0,C,incompetentes cuidam povo brasileiro poucos re...
2,@_carmeloneto \nOs 'cumpanhero' quebraram toda...,0,1,A,0.0,B,0,E,cumpanhero quebraram toda regras
3,@_GlitteryKisses é isso não conseguem pensar n...,0,0,C,0.0,V,0,D,conseguem pensar sentido lato além vê frente o...
4,@_iglira bom dia macaco branco haha,1,0,A,1.0,I,1,E,bom dia macaco branco haha
...,...,...,...,...,...,...,...,...,...
5665,@zecarlosantos2 é o unico que nao se corrompe....,0,1,C,0.0,B,0,A,unico nao corrompenao vende chega aroporto apl...
5666,"@zqkitowz sei das cotas, mas não sabia disso, ...",1,1,D,1.0,It,0,A,sei cotas sabia disso putaria porra
5667,"@zqkitowz sim, a maioria do eleitorado é mulhe...",0,0,C,0.0,V,0,C,sim maioria eleitorado mulher
5668,"@zurcju seguir no tt é facíl, apresentar as am...",1,1,C,1.0,S,0,A,seguir tt facíl apresentar amigas sapatão ngm ...


# Word Embedding

## GloVe

In [5]:
GLOVE_MODEL_FILE = './dataset/glove.twitter.27B/glove.twitter.27B.100d.txt'
max_len = 128
embedding_dim = 100

# Tokenize
token = Tokenizer()
token.fit_on_texts(df['pre_processed_text'])
seq = token.texts_to_sequences(df['pre_processed_text'])

# Padding
pad_seq = pad_sequences(seq,maxlen=embedding_dim)

# Vocab size
vocab_size = len(token.word_index)+1

# Load embedding vector
embedding_vector = {}
f = open(GLOVE_MODEL_FILE)
for line in tqdm(f):
    value = line.split(' ')
    word = value[0]
    coef = np.array(value[1:],dtype = 'float32')
    embedding_vector[word] = coef

1193514it [00:11, 102568.63it/s]


In [6]:
# Keep a out of vocabullary dict
oov_dict = {}

# Generate embedding matrix
embedding_matrix = np.zeros((vocab_size,embedding_dim))
for word,i in tqdm(token.word_index.items()):
    embedding_value = embedding_vector.get(word)
    if embedding_value is not None:
        embedding_matrix[i] = embedding_value
    else:
        oov_dict[word] = np.random.uniform(-1., 1., (embedding_dim,)) # Generate new random vector
        embedding_matrix[i] = oov_dict[word]


# Transform text into embed vector
embedded_sequences = np.zeros((len(pad_seq), max_len, embedding_dim))
for i, seq in enumerate(pad_seq):
    for j, idx in enumerate(seq):
        if idx > 0:  # Skip padding index
            embedded_sequences[i, j] = embedding_matrix[idx]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15340/15340 [00:00<00:00, 528239.47it/s]


## Bag of Words

In [7]:
def bag_of_words(X_train, X_test, n_grams):
    vectorizer = CountVectorizer(ngram_range=(1, n_grams))
    X_train = vectorizer.fit_transform(X_train).toarray()
    X_test = vectorizer.transform(X_test).toarray()
    return X_train,X_test

# Split into training and test sets

In [8]:
# X = df[pre_processed_text]
X = embedded_sequences
y = df['hatespeech_comb']

# Flatten
X = np.array([matrix.ravel() for matrix in X])

RANDOM_STATE = 42

# Hold out
sss = StratifiedShuffleSplit(n_splits = 1, test_size = 0.2, train_size = 0.8, random_state = RANDOM_STATE)
for i, (train_index, test_index) in enumerate(sss.split(X, y)):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

# Training

## MLP

In [9]:
activation_func = 'relu'
epochs = 10
batch_size = 128
learning_rate = 0.001

def mlp(learning_rate=0.001, activation_func='relu', clipvalue=0.5):
    model = Sequential()
    model.add(InputLayer(shape=(max_len*embedding_dim,)))
    model.add(Dense(units = 1000, activation = activation_func, kernel_initializer = 'random_uniform'))
    model.add(Dropout(0.2))
    model.add(Dense(units = 500, activation = activation_func, kernel_initializer = 'random_uniform'))
    model.add(Dropout(0.2))
    model.add(Dense(units = 1, activation = 'sigmoid'))
    
    adam = Adam(learning_rate = learning_rate, clipvalue = clipvalue)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['binary_accuracy'])    
    return model

In [10]:
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2, train_size=0.8, random_state=42)

results = []
f1 = []
accuracy = []

X = embedded_sequences
y = df['hatespeech_comb']

X = np.array([matrix.ravel() for matrix in X])

print("# Training")

for i, (train_index_cv, val_index) in enumerate(sss.split(X_train, y_train)):
    print(f"Folder :{i}")
    X_train_cv, X_val = X[train_index_cv], X[val_index]
    y_train_cv, y_val = y[train_index_cv], y[val_index]

    
    # Define EarlyStopping callback
    early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)

    # Model
    model = KerasClassifier(model=mlp,
                        epochs=epochs,
                        batch_size=batch_size,
                        callbacks=[early_stopping])
    
    # Fit
    model.fit(X_train_cv, y_train_cv)
    pred = model.predict(X_val)

    result = classification_report(y_val, pred)
    results.append(result)

    f = f1_score(y_val, pred)
    f1.append(f)
    print(f"# F1: {f}")

    acc = accuracy_score(y_val, pred)
    accuracy.append(acc)
    print(f"# Accuracy: {acc}")
    print("===============")

print("# Mean Accuracy: ", mean(accuracy))
print("# Mean F1: ", mean(f1))

# Training
Folder :0
Epoch 1/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - binary_accuracy: 0.6653 - loss: 0.6348
Epoch 2/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - binary_accuracy: 0.7892 - loss: 0.4351
Epoch 3/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - binary_accuracy: 0.9257 - loss: 0.2389
Epoch 4/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - binary_accuracy: 0.9715 - loss: 0.0983
Epoch 5/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - binary_accuracy: 0.9873 - loss: 0.0498
Epoch 6/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - binary_accuracy: 0.9958 - loss: 0.0259
Epoch 7/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - binary_accuracy: 0.9985 - loss: 0.0107
Epoch 8/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - binary_accuracy: 0.9980 - loss: 0.0086
Epoch 9/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - binary_accuracy: 0.9981 - loss: 0.0092
Epoch 10/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - binary_accuracy: 0.9990 - loss: 0.0095
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
# F1: 0.529980657640

# Evaluation

In [11]:
model = KerasClassifier(model = mlp,
                        epochs = epochs,
                        batch_size = batch_size,
                        callbacks=[early_stopping])

model.fit(pd.DataFrame(X_train.tolist()), y_train)
pred = model.predict(pd.DataFrame(X_test.tolist()))

result = classification_report(y_test, pred)
f1 = f1_score(y_test, pred)
accuracy = accuracy_score(y_test, pred)

print(result)

Epoch 1/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - binary_accuracy: 0.6369 - loss: 0.6837
Epoch 2/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - binary_accuracy: 0.7710 - loss: 0.4622
Epoch 3/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - binary_accuracy: 0.8902 - loss: 0.2815
Epoch 4/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - binary_accuracy: 0.9657 - loss: 0.1231
Epoch 5/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - binary_accuracy: 0.9843 - loss: 0.0604
Epoch 6/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - binary_accuracy: 0.9941 - loss: 0.0283
Epoch 7/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - binary_accuracy: 0.9950 - loss: 0.0207
Epoch 8/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - binary_accuracy: 0.9948 - loss: 0.0261
Epoch 9/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - binary_accuracy: 0.9980 - loss: 0.0140
Epoch 10/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - binary_accuracy: 0.9970 - loss: 0.0144
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
              precision    recall  f1-sco